In [1]:
import pandas as pd
import numpy as np
import config
import time
from numba import jit

In [2]:
def timestamp_datetime(value):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(value))

def time_feat(df,featList,featName):
    '''scaler = StandardScaler()
    tmp = df.groupby(featList).size().reset_index().rename(columns={0:featName})
    tmp[featName] = scaler.fit_transform(tmp[featName].values.reshape(-1,1))
    df = df.merge(tmp,'left',on=featList)'''
    df[featName] = df.groupby(featList)['context_timestamp'].rank(method='first')   
    return df

def process(df):
    df['time'] = df.context_timestamp.apply(timestamp_datetime)
    df['day'] = df.time.apply(lambda x: int(x[8:10]))
    df['hour'] = df.time.apply(lambda x: int(x[11:13]))
    '''for lst in timeFeatList:
        df = time_feat(df,lst,'_'.join(lst))'''
    df['item_property_list_clean'] = df['item_property_list'].apply(lambda x:';'.join(sorted(set(str(x).split(';')))))
    for i in range(3):
        df['item_category_list_bin%d'%i] = df['item_category_list'].apply(lambda x: x.split(';')[i] if len(x.split(';'))>i else -1)
    df["missing_feat"] = np.sum((df == -1).values, axis=1)
    return df

In [3]:
def focus_one_record(df,key_var=[],time_diff=False):
    nameBase = '_'.join(key_var)
    ###当天前后的数据情况
    df[nameBase+'_before_cnt'] = df.groupby(key_var+['day'])['context_timestamp'].rank(method='min') - 1
    df[nameBase+'_after_cnt'] = df.groupby(key_var+['day'])['context_timestamp'].rank(method='min',ascending=False)- 1
    df[nameBase+'_sametime_cnt'] = df.groupby(key_var+['day'])['context_timestamp'].rank(method='max') - df.groupby(key_var+['day'])['context_timestamp'].rank(method='min')+1
    df = df.merge(df.groupby(key_var+['day'],as_index=False)['context_timestamp'].agg({nameBase+'_day_cnt':'count'}),'inner',key_var+['day'])
    for feat in ['_before_cnt','_after_cnt','_sametime_cnt']:
        df[nameBase+feat+'_ratio'] = df[nameBase+feat]*1.0/df[nameBase+'_day_cnt']
        del df[nameBase+feat]
        
    ###前一天购买/浏览的数量
    '''dfTmp =  df.groupby(key_var+['day'],as_index=False)['is_trade'].agg({nameBase+'_preday_trade_cnt':'sum',nameBase+'_preday_cnt':'count'})
    dfTmp['day'] = dfTmp['day']+1
    df = df.merge(dfTmp,'left',key_var+['day'])
    df[nameBase+'_preday_trade_ratio'] = df[nameBase+'_preday_trade_cnt']*1.0/df[nameBase+'_preday_cnt']
    for feat in ['_preday_trade_cnt','_preday_cnt','_preday_trade_ratio']:
        df[nameBase+feat].fillna(0,inplace=True)'''
        
    if time_diff:
        ###广告展示上下间隔
        dfTmp = df[[nameBase+'_before_cnt',nameBase+'_after_cnt',nameBase+'_sametime_cnt','time']+key_var+['day']]
        dfTmp.rename(columns={'time':'new_time'},inplace=True)
        dfTmp['next_record'] = dfTmp[nameBase+'_before_cnt'] + dfTmp[nameBase+'_sametime_cnt'] + 1
        dfTmp['last_record'] = dfTmp[nameBase+'_after_cnt'] + dfTmp[nameBase+'_sametime_cnt'] + 1
        df = df.merge(dfTmp[key_var+['day','next_record','new_time']],'left',left_on = key_var+['day',nameBase+'_before_cnt'],right_on = key_var+['day','next_record'])
        df['next_time_dur'] = (pd.to_datetime(df['time'])-pd.to_datetime(df['new_time'])).dt.seconds
        df['next_time_dur'].fillna(999999,inplace=True)
        df.loc[df[nameBase+'_sametime_cnt']>1,'next_time_dur'] = 0
        del df['new_time']
        del df['next_record']

        df = df.merge(dfTmp[key_var+['day','last_record','new_time']],'left',left_on = key_var+['day',nameBase+'_after_cnt'],right_on = key_var+['day','last_record'])
        df['last_time_dur'] = (pd.to_datetime(df['new_time'])-pd.to_datetime(df['time'])).dt.seconds
        df['last_time_dur'].fillna(999999,inplace=True)
        df.loc[df[nameBase+'_sametime_cnt']>1,'last_time_dur'] = 0
        del df['new_time']
        del df['last_record']
    return df    
    

In [4]:
def Discretize(vlu, list_cat, greedy_f=False):
    if len(list_cat) == 0:
        return set([str(vlu)])
    cut_l = ['-Inf'] + [str(i) for i in list_cat if i < vlu]
    cut_h = [str(i) for i in list_cat if i >= vlu] + ['Inf']
    if greedy_f:
        return set([i + '_' + j for i in cut_l for j in cut_h])
    else:
        return set([cut_l[-1] + '_' + cut_h[0]])
    
def Discretization(rec, dict_cat, greedy_f=False):
    return (dict([(key, Discretize(rec[key], dict_cat[key], greedy_f)) for key in dict_cat]))

def cat_str(rec, dict_cat, greedy_f=False):
    cat_tag = Discretization(rec, dict_cat, greedy_f=False)
    flat_list = ';'.join([key + '_' + str(item) for key in cat_tag.keys() for item in cat_tag[key]])
    return (flat_list)

@jit
def _agg_df(df, grp_key, sum_var, cnt_var, stat_var):
    grouped = df.groupby(grp_key)
    agg_sum = grouped[sum_var].agg('sum').reset_index().melt(id_vars=grp_key, value_vars=sum_var)
    agg_sum['method'] = 'sum'
    agg_uniq = grouped[cnt_var].agg('nunique').reset_index().melt(id_vars=grp_key, value_vars=cnt_var)
    agg_uniq['method'] = 'cnt'
    agg_max = grouped[stat_var].agg('max').reset_index().melt(id_vars=grp_key, value_vars=stat_var)
    agg_max['method'] = 'max'
    agg_min = grouped[stat_var].agg('min').reset_index().melt(id_vars=grp_key, value_vars=stat_var)
    agg_min['method'] = 'min'
    agg_mst = pd.concat([agg_sum, agg_uniq, agg_max, agg_min])
    return (agg_mst)

def _sig_calc(cdr_rec, key_var, TS_cat, dict_cat,sum_var, cnt_var, stat_var):
    cdr_dict = cdr_rec.to_dict(orient='records')
    cdr_dict = [dict(list(x.items()) + [('TS', '~'.join(list(Discretization(x, TS_cat).values())[0]))] + [
        ('cat_str', cat_str(x, dict_cat))]) for x in cdr_dict]
    cdr_rec = pd.DataFrame(cdr_dict).reset_index()
    # one to many
    cat_driver = pd.concat([pd.Series(row['index'], row['cat_str'].split(';'))
                            for _, row in cdr_rec.iterrows()]).reset_index()
    cat_driver.columns = ['cat', 'index']
    cdr_rec_m = pd.merge(cdr_rec, cat_driver, on='index', how='inner')
    # Aggregation
    if key_var=='':
        key_var_tmp = ['TS']
    else:
        key_var_tmp = key_var+['TS']
    agg_mst_m = _agg_df(cdr_rec_m, key_var_tmp+['cat'], sum_var, cnt_var, stat_var)
    agg_mst_m['var_name'] = agg_mst_m[['method', 'variable', 'cat']].apply(
        lambda x: x[0] + '_' + x[1] + '_cat_' + str(x[2]), axis=1)
    if key_var=='':
        agg_mst_m['key'] = agg_mst_m[key_var_tmp]
    else:
        agg_mst_m['key'] = agg_mst_m[key_var_tmp].apply(lambda x: '{0}:{1}'.format(x[0], x[1]), axis=1)
    agg_df_m = agg_mst_m.pivot(index='key', columns='var_name', values='value').reset_index()

    agg_mst = _agg_df(cdr_rec, key_var_tmp, sum_var, cnt_var, stat_var)
    agg_mst['var_name'] = agg_mst[['method', 'variable']].apply(lambda x: x[0] + '_' + x[1] + '_cat_total_all', axis=1)
    if key_var=='':
        agg_mst['key'] = agg_mst[key_var_tmp]
    else:
        agg_mst['key'] = agg_mst[key_var_tmp].apply(lambda x: '{0}:{1}'.format(x[0], x[1]), axis=1)
    agg_df = agg_mst.pivot(index='key', columns='var_name', values='value').reset_index()

    agg_df = pd.merge(agg_df, agg_df_m, on='key', how='left')
    
    sig_ratio = [sig for sig in agg_df.columns if
                 (('sum_' in sig) or ('cnt_' in sig)) and ('cat_total_all' not in sig)]
    for sig in sig_ratio:
        agg_df['rto_' + sig] = agg_df[[sig, sig.split('cat')[0] + 'cat_total_all']].apply(
            lambda x: (x[0] + 0.0) / x[1], axis=1)
        del agg_df[sig]
    return agg_df

In [18]:
def _offline_feat(df,key_var='user_id',stat_var=[],part_var=[],mean_var=[],train_feat_col=None):
    if not isinstance(key_var,list):
        key_var = [key_var]
    left_key = key_var.copy()
    if not train_feat_col:
        key_var.append(train_feat_col[1])
        left_key.append(train_feat_col[0])
        
    base_name = '~'.join(key_var)
    df = df.merge(df.groupby(key_var,as_index=False)['instance_id'].agg({base_name+'_cnt':'count'}),'left',left_on =left_key, right_on = key_var)
    df = df.merge(df.groupby(key_var,as_index=False)['is_trade'].agg({base_name+'_trade_cnt':'sum',base_name+'_trade_ratio':'mean'}),'left',left_on =left_key, right_on = key_var)
    df[base_name+'_notrade_cnt'] = df[base_name+'_cnt']-df[base_name+'_trade_cnt']
    dfTmp = df.loc[df['is_trade']==1]
    for stat in stat_var:
        df = df.merge(df.groupby(key_var,as_index=False)[stat].agg({base_name+'_'+stat+'_min':'min',base_name+'_'+stat+'_max':'max'}),'left',left_on =left_key, right_on = key_var)    
    for part in part_var:
        df = df.merge(df.groupby(key_var,as_index=False)[part].agg({base_name+'_'+part+'_cnt':'nunique'}),'left',left_on =left_key, right_on = key_var)
        df = df.merge(dfTmp.groupby(key_var,as_index=False)[part].agg({base_name+'_'+part+'_trade_cnt':'nunique'}),'left',left_on =left_key, right_on = key_var)
        df[base_name+'_'+part+'_trade_cnt'].fillna(0,inplace=True)
        df[base_name+'_'+part+'_trade_ratio'] = 1.0*df[base_name+'_'+part+'_trade_cnt']/df[base_name+'_'+part+'_cnt']
    for var in mean_var:
        df = df.merge(df.groupby(key_var+[var],as_index=False)['is_trade'].sum().groupby(key_var,as_index=False)['is_trade'].agg({base_name+'_'+var+'_avg_trade':'mean'}),'left',left_on =left_key, right_on = key_var)
    return df
    
def 

In [31]:
def map_col(df):
    map_dict = {'item_price_level':[4,5,6,7,8,9],
     'item_sales_level':[4,6,9,10,11,12,13,14,16],
     'item_pv_level':[6,9,10,11,12,13,14,15,16,17,18,19,20],
     'user_age_level':[1001,1002,1003,1004,1005],
     'context_page_id':[4001,4002,4004,4006,4008,4010,4013,4016,4018],
     'shop_review_num_level':[5,9,14,15,16,17,18,20,21],'user_occupation_id':{-1:2003},'user_star_level':{-1:3000}
    }
    for key,value in map_dict.items():
        if isinstance(value,list):
            df[key+'_mapped'] = 0
            for i in range(1,len(value)):
                df.loc[df[key]<=value[i],key+'_mapped'] = i
        else:
            '''df[key+'_mapped'] = df[key]
            for key_sub,value_sub in value.items():
                df.loc[df[key]==key_sub,key+'_mapped'] = value_sub'''
            df[key+'_mapped'] = df[key].apply(lambda x:value.get(x,x))
            
    return df
            

In [32]:
map_col(dfAll)

,context_id,context_page_id,context_timestamp,day,hour,instance_id,is_trade,item_brand_id,item_category_list,item_category_list_bin0,...,item_id_shop_id_trade_cnt,item_id_shop_id_trade_ratio,item_price_level_mapped,item_sales_level_mapped,item_pv_level_mapped,user_age_level_mapped,context_page_id_mapped,shop_review_num_level_mapped,user_occupation_id_mapped,user_star_level_mapped
0,282924576738839389,4006,1537236544,18,10,108641074714126964,0.0,1975590437749032870,7908382889764677758;5799347067982556520,7908382889764677758,...,0.0,0.0,5,8,12,4,8,8,2005,3003
1,4007979028023783431,4001,1537243232,18,12,5754713551599725161,0.0,1975590437749032870,7908382889764677758;5799347067982556520,7908382889764677758,...,0.0,0.0,5,8,12,4,8,8,2005,3006
2,4884875192608989870,4001,1537211052,18,3,842679481291040981,0.0,1975590437749032870,7908382889764677758;5799347067982556520,7908382889764677758,...,0.0,0.0,5,8,12,4,8,8,2005,3004
3,840119421106178602,4016,1537222670,18,6,937088850059189027,0.0,1975590437749032870,7908382889764677758;5799347067982556520,7908382889764677758,...,0.0,0.0,5,8,12,4,8,8,2005,3006
4,1736769971710354684,4001,1537271320,18,19,7975697065017708072,0.0,1975590437749032870,7908382889764677758;5799347067982556520,7908382889764677758,...,0.0,0.0,5,8,12,4,8,8,2005,3001
5,4434980272230296456,4003,1537282855,18,23,7764762765372067286,0.0,1975590437749032870,7908382889764677758;5799347067982556520,7908382889764677758,...,0.0,0.0,5,8,12,4,8,8,2005,3002
6,3622211816051289512,4001,1537280317,18,22,6956333474094867789,0.0,9057103201734987852,7908382889764677758;8277336076276184272,7908382889764677758,...,1.0,1.0,5,8,12,0,8,8,2005,3000
7,7851031132945961016,4001,1537261120,18,16,8387099821892927911,0.0,9057103201734987852,7908382889764677758;8277336076276184272,7908382889764677758,...,1.0,1.0,5,8,12,4,8,8,2002,3002
8,8388974876851097582,4001,1537208871,18,2,4021878205550012615,0.0,5520678735822176314,7908382889764677758;5755694407684602296,7908382889764677758,...,1.0,1.0,5,8,12,4,8,8,2002,3007
9,1138535512266486347,4003,1537285390,18,23,6499571365974135517,0.0,9057103201734987852,7908382889764677758;8277336076276184272,7908382889764677758,...,1.0,1.0,5,8,12,4,8,8,2005,3006


In [19]:
dfTrain = pd.read_table(config.TRAIN_FILE,sep=' ')
dfTrain.drop_duplicates(inplace=True)
dfTrain.reset_index(inplace=True,drop =True)
dfTest = pd.read_table(config.TEST_FILE,sep=' ')

dfTrain = process(dfTrain)
dfTest = process(dfTest)

dfAll = pd.concat([dfTrain,dfTest],axis=0)
dfAll.reset_index(inplace=True,drop=True)
trainNum = dfTrain.shape[0]
dfAll['cnt_rec'] = 1
dfAll.shape

(496482, 36)

In [20]:
dfAll['feat_set'] = dfAll['day'] + 1

In [21]:
keyList = ['user_id','shop_id','item_id']
partList = [
    ['item_id','shop_id'],
    ['user_id','item_id'],
    ['user_id','shop_id']
]
meanList = [
    ['shop_id'],
    ['item_id'],
    []
]
for i in range(3):
    keyVar = keyList[i]
    partVar = partList[i]
    meanVar = meanList[i]
    statVar = []
    for key,value in config.STAT_DICT.items():
        if key==keyVar:
            continue
        statVar += value
    dfAll = _offline_feat(dfAll,keyVar,statVar,partVar,meanVar,['day','feat_set'])

In [23]:
dfAll.isnull().sum()

context_id                                   0
context_page_id                              0
context_timestamp                            0
day                                          0
hour                                         0
instance_id                                  0
is_trade                                 18371
item_brand_id                                0
item_category_list                           0
item_category_list_bin0                      0
item_category_list_bin1                      0
item_category_list_bin2                      0
item_city_id                                 0
item_collected_level                         0
item_id                                      0
item_price_level                             0
item_property_list                           0
item_property_list_clean                     0
item_pv_level                                0
item_sales_level                             0
missing_feat                                 0
predict_categ

In [ ]:
for varKey in ['use_id','shop_id','item_id']: 
    tmpFeat = _sig_calc(daAll,[varKey],{'day':[18,19,20,21,22,23,24,25]},{'item_sales_level':[],'item_collected_level':[]},['user_gender_id'],['cnt_ind'],['cnt_ind'])

In [ ]:
sum_var = ['cnt_rec']
cnt_var = ['item_price_level','item_sales_level','item_collected_level','item_pv_level','user_age_level','user_occupation_id','context_page_id','shop_review_num_level','shop_star_level','hour']
stat_var = ['cnt_rec']

In [ ]:
dict_cat = {'cnt_rec':[],'item_price_level':[],'item_sales_level':[],'item_collected_level':[],'item_pv_level':[],'hour':[6,12,20]}
TS_cat = {'day':[18,19,20,21,22,23,24,25]}
key_var = ['user_id']

In [ ]:
_sig_calc(dfAll,['user_id'],{'day':[18,19,20,21,22,23,24,25]},dict_cat,sum_var,cnt_var,stat_var)

In [ ]:
cdr_rec = dfAll
cdr_dict = cdr_rec.to_dict(orient='records')
cdr_dict = [dict(list(x.items()) + [('TS', '~'.join(list(Discretization(x, TS_cat).values())[0]))] + [
    ('cat_str', cat_str(x, dict_cat))]) for x in cdr_dict]
cdr_rec = pd.DataFrame(cdr_dict).reset_index()
# one to many
cat_driver = pd.concat([pd.Series(row['index'], row['cat_str'].split(';'))
                        for _, row in cdr_rec.iterrows()]).reset_index()
cat_driver.columns = ['cat', 'index']
cdr_rec_m = pd.merge(cdr_rec, cat_driver, on='index', how='inner')
# Aggregation
if key_var=='':
    key_var_tmp = ['TS']
else:
    key_var_tmp = key_var+['TS']
agg_mst_m = _agg_df(cdr_rec_m, key_var_tmp+['cat'], sum_var, cnt_var, stat_var)
agg_mst_m['var_name'] = agg_mst_m[['method', 'variable', 'cat']].apply(
    lambda x: x[0] + '_' + x[1] + '_cat_' + str(x[2]), axis=1)
if key_var=='':
    agg_mst_m['key'] = agg_mst_m[key_var_tmp]
else:
    agg_mst_m['key'] = agg_mst_m[key_var_tmp].apply(lambda x: '{0}:{1}'.format(x[0], x[1]), axis=1)

In [ ]:
agg_mst_m

In [ ]:
agg_mst_m.pivot(index='key', columns='var_name', values='value')

In [ ]:
df = pd.DataFrame({'A':[1,2,3],'B':['a;b','a;b','a;b;a']})

In [ ]:
df['B'].apply(lambda x:';'.join(sorted(set(str(x).split(';')))))

In [ ]:
dfTrain = pd.read_table(config.TRAIN_FILE,sep=' ')
dfTrain.drop_duplicates(inplace=True)
dfTrain.reset_index(inplace=True,drop =True)
dfTest = pd.read_table(config.TEST_FILE,sep=' ')

dfTrain = process(dfTrain)
dfTest = process(dfTest)

In [ ]:
dfAll = pd.concat([dfTrain,dfTest])
dfAll.reset_index(inplace=True,drop=True)
dfAll.shape

In [ ]:
Test = focus_one_record(dfAll,['user_id'])

In [ ]:
Test.dtypes

In [ ]:
dfTest['next_time_dur'].dt.days.max()

In [ ]:
dfTest['next_time_dur'].map(lambda x: x/np.timedelta64(1, 's'))

In [ ]:
dfTest.dtypes

In [ ]:
df = dfAll
key_var = ['user_id']

In [ ]:
pd.to_datetime(dfAll['time'])

In [ ]:
nameBase = '_'.join(key_var)
###当天前后的数据情况
df[nameBase+'_before_cnt'] = df.groupby(key_var+['day'])['context_timestamp'].rank(method='min')

In [ ]:
df.groupby('user_id')['context_timestamp'].rank(method='first')

In [ ]:
df.groupby('user_id')['context_timestamp'].rank(method='min')

In [ ]:
df.groupby(key_var+['day'])['context_timestamp'].count()

In [ ]:
key_var+['day']